# Week 2: Fine-Tuning the Pretrained Model

**Objective:** To improve performance by unfreezing parts of the base model and training them with a low learning rate.

This notebook covers:
- Loading the model from the feature extraction phase.
- Unfreezing the top layers of the MobileNetV2 base.
- Continuing training (fine-tuning) with a very low learning rate.

In [ ]:
import os
import sys
import tensorflow as tf
from tensorflow import keras

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", "..")))
from config import *
from Week1_Data_and_Baseline.utils.data_utils import create_data_generators
from Week2_Transfer_Learning.utils.model_utils import unfreeze_layers

In [ ]:
train_ds, val_ds = create_data_generators(TRAIN_DIR, VAL_DIR, IMG_SIZE, BATCH_SIZE, RANDOM_SEED)

In [ ]:
# Load the model from Phase 1
fine_tuning_model = keras.models.load_model(get_model_path("mobilenetv2", "phase1"))

# Unfreeze some layers
fine_tuning_model = unfreeze_layers(fine_tuning_model, num_layers_to_unfreeze=30)

In [ ]:
fine_tuning_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_TRANSFER_PHASE2), # Lower LR!
    loss=LOSS_FUNCTION,
    metrics=METRICS
)

history_phase2 = fine_tuning_model.fit(
    train_ds,
    epochs=EPOCHS_TRANSFER_PHASE1 + EPOCHS_TRANSFER_PHASE2, # Total epochs
    initial_epoch=EPOCHS_TRANSFER_PHASE1, # Start from where we left off
    validation_data=val_ds,
    callbacks=[keras.callbacks.ModelCheckpoint(get_model_path("mobilenetv2", "final"), save_best_only=True)]
)